# Preparation

In [1]:
import sys, os, glob
import singlepulse as sp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib as mpl
from tkinter.filedialog import askopenfilenames
from datetime import datetime

mpl.rcParams['text.usetex'] = True
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = ['Computer Modern'] # oder eine andere LaTeX-Schriftart
mpl.rcParams['text.latex.preamble'] = r'\usepackage{amsmath}'
mpl.rcParams['axes.titlesize'] = 14
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12
mpl.rcParams['legend.fontsize'] = 12
mpl.rcParams['figure.titlesize'] = 20
mpl.rcParams['figure.labelsize'] = 14

In [ ]:
selected_files = askopenfilenames(title="Wähle .csv Files zum Bearbeiten aus", filetypes=[("CSV Files", "*.csv")])

selected_files
print("Test")

('/Users/eliasfoisner/Studium/TCH Master/LU Masterarbeit/Data/241022 FourElement Oxidrate + Calibration/Regular/Particles/Eu153 CeO156 Probe4 50Par 15% 10x10 7.csv',)

# Plot many files

In [4]:
objects = dict()
results = []
ratios = []

for file in selected_files:
    f = os.path.basename(file)
    objects[f] = sp.SinglePulse(file)
    isotopes = objects[f].measured_isotopes.tolist()

    for i in isotopes:
        criterion, threshold = 10, 50
        if i == "Eu/153":
            criterion, threshold = 10, 40
        if i == "EuO/169":
            criterion, threshold = 1, 5
        objects[f].timescale(i, cycle_time=50e-6)
        objects[f].savgol(i)
        objects[f].peak_finding(i, threshold=threshold, distance=100e-3)
        objects[f].peak_width(i, criterion=criterion)
        objects[f].peak_area(i, resize=1.5)
        objects[f].plot(i, peaks=True, integration=True)

        results.append(
            {
                'species': i,
                'no. of peaks': len(objects[f].peaks[i]),
                f'peak width ({criterion}% criterion)': objects[f].peaks[i]['width'].mean() * 1e6,
                'peak width stdev': objects[f].peaks[i]['width'].std() * 1e6,
                f'peak height': objects[f].peaks[i]['height'].mean(),
                'peak height stdev': objects[f].peaks[i]['height'].std(),
                'peak area': objects[f].peaks[i]['area'].mean(),
                'peak area stdev': objects[f].peaks[i]['area'].std()
            }
        )
        print(f"Done: {f}")

    #ratios.append(objects[f].area_ratio("Eu/153", "EuO/169"))

#ratios = pd.concat(ratios)
#ratios = ratios.reset_index(drop=True).squeeze()
#print(ratios.mean(), ratios.std())

results = pd.DataFrame(results)
#area_euo = results[results["species"] == "EuO/169"]["peak area"].mean()
#area_eu = results[results["species"] == "Eu/153"]["peak area"].mean()
#ratio = area_euo / (area_euo + area_eu)
print(results)
#print(ratio)


#results = results.apply(pd.to_numeric, errors='ignore')
#results = results.dropna(axis=1, how='all')
#results = results.fillna(0)
#results.to_excel('output.xlsx')
#print(results)

sp.SinglePulse.fig.show_dash()

Done: Eu153 CeO156 Probe4 50Par 15% 10x10 7.csv
Done: Eu153 CeO156 Probe4 50Par 15% 10x10 7.csv
   species  no. of peaks  peak width (10% criterion)  peak width stdev  \
0   Eu/153            29                 3524.019941        226.304172   
1  CeO/156            29                 3539.637305        272.478685   

   peak height  peak height stdev   peak area  peak area stdev  
0   115.218679          25.878967  273.556973        65.482305  
1   100.015307          22.165754  231.876452        53.431487  


Address already in use
Port 8050 is in use by another program. Either identify and stop that program, or start the server with a different port.


SystemExit: 1

/Users/eliasfoisner/PythonProjects/singlepulse/venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3557: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.

